<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/502_LLM_(%EC%8B%A4%EC%8A%B5)_LangChain%EA%B3%BC_RAG%EB%A1%9C_%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0_%EC%97%AC%EB%9F%AC%ED%8C%8C%EC%9D%BC%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LangChain과 RAG로 챗봇 만들기**




---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---

## **실습 환경**

- **구글 코랩 + OpenAI API Key**


In [1]:
# 설치된 특정 라이브러리 버전 확인
%pip list | grep -E 'langchain|tiktoken|openai|pypdf|chromadb'

langchain                                0.3.27
langchain-core                           0.3.79
langchain-text-splitters                 0.3.11
openai                                   1.109.1
tiktoken                                 0.12.0


In [2]:
!pip install -q langchain langchain-openai openai tiktoken chromadb langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.6 MB/s eta 0:0

In [3]:
# 설치된 특정 라이브러리 버전 확인
%pip list | grep -E 'langchain|tiktoken|openai|pypdf|chromadb'

chromadb                                 1.2.2
langchain                                0.3.27
langchain-community                      0.3.31
langchain-core                           0.3.79
langchain-openai                         0.3.35
langchain-text-splitters                 0.3.11
openai                                   1.109.1
tiktoken                                 0.12.0



---



# **여러 문서에서 찾아서 답변하는 챗봇 만들기**

## **OpenAI API Key 사용-LangChain 설정**

- **OpenAI API Key 발급**
    - https://platform.openai.com/account/api-keys 이동
        - **[+Create new secret key]** 클릭
        - name에 키 이름 넣고**(예: test_key)**
        - **[Copy]** 클릭 키를 메모장에 복사해 놓는다. --> 이 키로 과금이 된다.
        - **[View user API keys]** 클릭
        - 좌측 메뉴에 **Billing** 선택
        - **[Add payment details]** 선택 --> **카드 등록, 5$ 결재**

In [4]:
import os

# os.environ["OPENAI_API_KEY"] = "여러분들의 Key 값"
os.environ['OPENAI_API_KEY'] =  ""


## **데이터 준비**

In [5]:
!wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip

--2025-10-27 01:19:00--  https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip [following]
--2025-10-27 01:19:00--  https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 835816 (816K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 816.23K  --.-KB/s    in 0.03s   

2025-10-27 01:19:00 (24.0 MB/s) - ‘data.zip’ saved [835816/835816]



In [6]:
# 57개 txt 데이터
!unzip data

Archive:  data.zip
  inflating: 1.txt                   
  inflating: 10.txt                  
  inflating: 11.txt                  
  inflating: 12.txt                  
  inflating: 13.txt                  
  inflating: 14.txt                  
  inflating: 15.txt                  
  inflating: 16.txt                  
  inflating: 17.txt                  
  inflating: 18.txt                  
  inflating: 19.txt                  
  inflating: 2.txt                   
  inflating: 20.txt                  
  inflating: 21.txt                  
  inflating: 22.txt                  
  inflating: 23.txt                  
  inflating: 24.txt                  
  inflating: 25.txt                  
  inflating: 26.txt                  
  inflating: 27.txt                  
  inflating: 28.txt                  
  inflating: 29.txt                  
  inflating: 3.txt                   
  inflating: 30.txt                  
  inflating: 31.txt                  
  inflating: 32.txt            

- **사용할 라이브러리**

In [7]:
# 데이터 로드 및 문서 처리
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

# 텍스트 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
from collections import Counter

# 임베딩 및 벡터DB화
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 질의응답: 챗봇
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI


## **데이터 로드 및 문서처리**

- **DirectoryLoader**()는 디렉토리 내의 모든 문서들을 로드
    - glob는 해당 디렉토리 내의 어떤 확장자를 가진 파일들을 로드할 것인지를 결정.
    - ex: glob="*.txt" (현재 경로에 있는 모든 txt 파일을 로드)
- txt 파일의 본문은 documents라는 파이썬 리스트의 각 원소로 할당

- **txt 문서**

In [8]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

# txt 문서
loader = DirectoryLoader(
    path='.', # 경로 안에 파일이 여러개라서 그냥 .만 사용
    glob="*.txt",
    loader_cls=TextLoader
)
documents = loader.load()
print('문서의 개수 :', len(documents))

문서의 개수 : 57


- **pdf 문서**

In [9]:
# # pdf 문서
# loader = DirectoryLoader(
#     path='.',
#     glob="*.pdf",
#     loader_cls=PyPDFLoader
# )
# documents = loader.load()
# print('문서의 개수 :', len(documents))

- **파이썬 코드**

In [10]:
# # **/*.py 문서
# loader = DirectoryLoader(
#     path='.',
#     glob="**/*.py",
#     loader_cls=TextLoader,
#     xclude=["**/__init__.py"]  # glob 패턴 외에 특정 파일을 제외할 수도 있다.
# )
# documents = code_loader.load()
# print('코드 문서의 개수 :', len(documents))

- **다양한 형식 문서** (doc 등) : UnstructuredFileLoader

In [ ]:
# # 다양한 형 문서 (doc 등)
# !pip install -qU unstructured

# from langchain_community.document_loaders import UnstructuredFileLoader

# loader = DirectoryLoader(
#     path='.',
#     glob="*.*", # 문서타입 전체
#     loader_cls=UnstructuredFileLoader,
#     use_multithreading=True # 여러 파일을 동시에 읽어와 속도 향상
# )
# # UnstructuredFileLoader는 지원하는 확장자만 필터링하여 로드합니다.
# docs = all_in_one_loader.load()
# print(f"총 {len(docs)}개의 다양한 문서를 로드했습니다.")

In [ ]:
# 문서 일부 출력
# for idx, doc in enumerate(documents):
#     if idx < 10:
#         print(f'{idx:2<} 번 문서 : {doc}')
#         print('-' * 50)

파이썬 리스트에 해당하는 documents에서 임의로 1번 문서와 21번 문서를 출력해보겠습니다.

In [11]:
print('1번 문서 :', documents[1]) # 실제 문서명의 순서와 동일하지 않음(ex. 1.txt와 documents[0]은 서로 다른 문서임)
print('-' * 20)
print('21번 문서 :', documents[2])
print('-' * 20)

1번 문서 : page_content='정책 이름: 창조적 예비청년창업가(로컬 크리에이터)
서울신용보증재단에서는 골목상권에서 지역 가치를 차별화된 콘텐츠로 구현하고 매력적인 도시 문화를 형성할 수 있는 로컬 크리에이터를 양성하기 위한 창업 지원 정책을 진행합니다. 이 프로그램은 5개월 동안 교육을 지원하며, 약 10개 팀 정도가 지원을 받을 수 있습니다. 신청자격은 19세에서 39세까지의 서울시 거주자이며, 로컬 브랜드 상권 2기 대상지에서 창업을 희망하는 예비 창업자여야 합니다. 학력, 전공, 취업 상태, 특화 분야에 대한 요건은 없습니다. 신청 방법, 신청 절차, 심사 및 발표 관련 사항, 제출 서류 등은 추후 공지를 통해 안내될 예정입니다. 이 프로그램은 서울신용보증재단에서 운영됩니다.' metadata={'source': '4.txt'}
--------------------
21번 문서 : page_content='정책제목: 청년인생설계학교
정책 유형은 "진로"이며, 해당 정책은 서울시청 미래청년기획단에서 주관하고 있습니다. 이 정책은 인생전환기에 있는 청년들에게 주체적인 경로 설정 기회를 제공하여 자존감 향상과 인적 역량 강화를 목표로 합니다. 지원 내용으로는 청년의 진로 발달 단계에 따라 맞춤형 자기이해 및 진로설계 프로그램을 제공합니다. 사업은 2023년 5월 1일부터 2023년 11월 30일까지 운영되며, 신청은 2023년 4월 6일부터 2023년 4월 18일까지 가능합니다. 총 1,060명을 지원할 예정이며, 자세한 내용은 "https://youth.seoul.go.kr/site/main/content/life_desi_sch_intro"에서 확인하실 수 있습니다. 신청 자격은 19세부터 39세까지 서울에 거주하는 청년이면 누구나 신청할 수 있습니다. 학력, 전공, 취업 상태, 특화 분야에는 제한이 없으며, 추가적인 단서나 제한 대상은 없습니다. 신청 방법은 청년몽땅정보통 온라인 신청을 통해 이루어지며, 심사 및 발표는 모집 시기마다 우선 및 

In [12]:
# 특정 문서의 내용
documents[0].page_content

'정책제목: 자립준비청년 자립수당 지급\n서울시 아동담당관에서는 아동복지시설에서 보호를 받다가 자립을 준비하고 있는 청년들에게 자립수당을 매월 40만원 지급하는 정책을 운영하고 있습니다. 이 정책은 아동복지시설이나 가정위탁에서 보호를 종료한 후 5년 이내의 청년들을 대상으로 하며, 매월 40만원을 지원합니다. 신청은 방문신청이나 우편신청을 통해 자치구에서 보장결정을 받은 후에 지급되며, 최대 5년 동안 받을 수 있습니다.\n\n사업 운영 기간은 2019년 1월 1일부터 2023년 12월 31일까지이며, 신청은 2023년 1월 1일부터 12월 31일까지 가능합니다. 지원 규모는 매월 40만원입니다.\n\n신청자격은 18세 이상부터 0세까지이며, 보호종료 후 5년 이내에 해당하는 청년들이 지원 대상입니다. 학력이나 전공 요건, 취업 상태에는 제한이 없습니다. 추가적으로 서비스 이용 청년은 19세부터 39세까지의 청년층을 의미합니다.\n\n자세한 신청 방법과 필요한 서류는 아동복지시설이 관할하는 읍, 면, 동 또는 본인의 주민등록 상 주소지 읍, 면, 동을 통해 신청하며, 자립수당 신청서, 금융교육 이수증, 보호종료 확인서 등의 서류를 제출해야 합니다.\n\n자세한 내용은 서울시청 미래청년기획단 사이트에서 확인하실 수 있습니다.'

In [18]:
# 특정 문서 metadata(문서명)
print("인덱스 0번 문서의 metadata: ", documents[0].metadata)
print("인덱스 0번 문서의 metadata의 source: ", documents[0].metadata['source'])

인덱스 0번 문서의 metadata:  {'source': '38.txt'}
인덱스 0번 문서의 metadata의 source:  38.txt


documents 리스트의 각 원소의 구조를 살펴보면 page_content의 값으로 본문이 들어가고, metadata 안의 source의 값으로 해당 문서의 원본 파일의 이름이 기재되어져 있습니다.

## **텍스트 분할**

ChatGPT API(GPT-3.5-turbo)는 입력으로 사용하는 텍스트와 ChatGPT API가 반환하는 답변의 길이를 모두 포함하여 최대 4,096 토큰을 처리할 수 있습니다. 예를 들어 ChatGPT API 입력으로 약 2,000 토큰 정도의 길이를 가진 입력을 사용한다면, ChatGPT는 약 2,000 토큰 내외의 답변을 할 수 있는 구조입니다. 입력의 길이가 지나치게 길 경우 ChatGPT API가 답변을 하던 도중 4,096개의 토큰을 모두 소진하면 더 이상 답변을 생성할 수 없어 텍스트가 중간에 끊긴 답변을 얻게 됩니다.  

이런 현상을 피하기 위해 너무 긴 문서들은 한 번에 입력으로 사용하는 것이 아니라 여러 개의 뭉치(Chunk)로 잘라서 ChatGPT API를 여러 번 호출하는 방식으로 해결하기도 합니다. Langchain의 RecursiveCharacterTextSplitter()는 이를 위해 사용하는 도구로 주어진 문서들을 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 합니다.

chunk_size는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap은 텍스트를 분할할 경우 텍스트의 내용을 얼만큼 겹치게 할 것인지를 의미합니다. 위에서 총 57개의 문서 파일을 로드한 documents를 입력으로 사용하여 분할된 텍스트 리스트를 texts에 저장합니다.

- **chunk_size 단위로 텍스트 분할**

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) # 서로가 같은 문서라는걸 이해하기위해 이정도까지는 overlap하겠다~ 이런의미
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수 :', len(texts))

분할된 텍스트의 개수 : 64


57개의 문서가 총 64개의 텍스트로 분할되었습니다. 그 중 임의로 첫번째 문서를 출력해보겠습니다.

In [15]:
texts[0]

Document(metadata={'source': '38.txt'}, page_content='정책제목: 자립준비청년 자립수당 지급\n서울시 아동담당관에서는 아동복지시설에서 보호를 받다가 자립을 준비하고 있는 청년들에게 자립수당을 매월 40만원 지급하는 정책을 운영하고 있습니다. 이 정책은 아동복지시설이나 가정위탁에서 보호를 종료한 후 5년 이내의 청년들을 대상으로 하며, 매월 40만원을 지원합니다. 신청은 방문신청이나 우편신청을 통해 자치구에서 보장결정을 받은 후에 지급되며, 최대 5년 동안 받을 수 있습니다.\n\n사업 운영 기간은 2019년 1월 1일부터 2023년 12월 31일까지이며, 신청은 2023년 1월 1일부터 12월 31일까지 가능합니다. 지원 규모는 매월 40만원입니다.\n\n신청자격은 18세 이상부터 0세까지이며, 보호종료 후 5년 이내에 해당하는 청년들이 지원 대상입니다. 학력이나 전공 요건, 취업 상태에는 제한이 없습니다. 추가적으로 서비스 이용 청년은 19세부터 39세까지의 청년층을 의미합니다.\n\n자세한 신청 방법과 필요한 서류는 아동복지시설이 관할하는 읍, 면, 동 또는 본인의 주민등록 상 주소지 읍, 면, 동을 통해 신청하며, 자립수당 신청서, 금융교육 이수증, 보호종료 확인서 등의 서류를 제출해야 합니다.\n\n자세한 내용은 서울시청 미래청년기획단 사이트에서 확인하실 수 있습니다.')

형식은 documents 원소에서 확인하였을 때와 동일하며 page_content에는 분할된 텍스트이 본문이 저장되어져 있고, source에는 해당 본문의 원본 파일의 이름이 저장되어져 있습니다. 57개의 문서가 64개의 텍스트로 분할되었다는 것은 일부 문서는 추가적으로 분할되었음을 의미합니다.  

metadata 안의 source의 값을 이용하여 어떤 문서들을 추가적으로 분할되었는지 확인해보겠습니다. texts의 각 원소들의 source 값을 모두 체크하였을 때 source 값이 중복되어 등장한다면 해당 문서는 추가적으로 분할되었음을 의미합니다.

- **2개 이상으로 분할된 문서**

In [19]:
source_lst = []
for i in range(0, len(texts)):
  source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서 :', filtered_counts)
print('분할된 텍스트의 개수 :', len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'49.txt': 2, '23.txt': 2, '48.txt': 2, '40.txt': 2, '36.txt': 2, '22.txt': 2, '31.txt': 2}
분할된 텍스트의 개수 : 64


확인 결과 48번 문서, 23번 문서, 31번 문서, 49번 문서, 36번 문서, 22번 문서, 40번 문서가 2개의 텍스트로 추가적으로 분할되었습니다. 총 57개의 문서 중 7개의 문서가 2개의 텍스트로 분할되었음으로 총 합 64개의 텍스트가 생기게 됩니다.

## **임베딩 및 벡터DB화**

앞서 6장에서 OpenAI Embedding API를 이용하여 텍스트를 임베딩하고, 코사인 유사도를 통해 유사한 텍스트를 가져오는 실습을 진행한 바 있습니다. Chroma DB는 이 과정들을 기능 별로 이미 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있도록 도와주는 편리한 벡터 응용 도구입니다. Chroma.from_documents()를 통해 벡터 도구 객체를 선언합니다. 이때 documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 사용하고, embedding에는 어떤 종류의 임베딩을 사용할 것인지를 기재해줍니다. 여기서는 6장에서와 같이 OpenAI의 Embedding API를 사용합니다.

In [20]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

/tmp/ipython-input-1011075910.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


## **벡터DB 검색(retriever)**

벡터 도구 객체를 선언하고 나면 as_retriever()를 통해서 입력된 텍스트로부터 유사한 텍스트를 찾아주는 retriever를 선언할 수 있습니다. retriever를 선언 후 get_relevant_documents()를 통해 입력된 텍스트와 유사한 문서들을 찾아서 반환합니다. 6장에서 실습했던 벡터의 유사도를 구하는 과정을 별도의 추가 구현없이 손쉽게 사용할 수 있습니다.

In [21]:
# 벡터DB 검색자
retriever = vectordb.as_retriever()

In [22]:
# 유사 문서 검색
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('--' * 20)
print('각 유사 문서의 문서 출처 :')
for doc in docs:
    print(doc.metadata["source"])

/tmp/ipython-input-1265528122.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")


유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.

이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.

이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.

이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': '23.txt'}
----------------------------------------
각 유사 문서의 문서 출처 :
23.txt
23.txt
40.txt
39.txt


### **결과를 k개 반환**

만약 유사한 문서의 개수를 임의로 지정하고 싶다면 retriever 선언 시에 search_kwargs={"k": <문서의 개수>}를 추가적으로 입력으로 넣어주면 됩니다. 다음의 코드는 유사한 텍스트가 2개만 출력되는 모습을 보여줍니다.

In [23]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [24]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")

for doc in docs:
    print(doc.metadata["source"])
    print(doc.page_content)
    print('--' * 20)

23.txt
이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.

이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.

이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.

이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.
----------------------------------------
23.txt
정책제목: 신혼부부 임차보증금 지원
정부에서는 주거 관련 정책을 통해 부담을 완화하여 더 나은 주거환경을 제공하고자 합니다. 현재 주거마련에 대한 부담으로 인해 혼인수가 감소하고 출산기피 현상이 발생하고 있습니다. 따라서 주거비 부담을 완화하여 이러한 문제를 해결하고, 좋은 주거환경을 제공하고자 합니다.

이 정책은 서울시청 주택정책과에서 주관하며, 주거 마련에 대한 부담을 완화하기 위한 내용을 포함하고 있습니다. 지원 대상은 관내 임차보증금 7억 이내의 주택 또는 주거용 오피스텔에 대해 해당하는 서울시민이나 서울로 전입 예정인 자입니다. 대출한도는 임차보증금의 90% 이내 또는 2억원 중 작은 

## **검색 체인 만들기**

이제 ChatGPT API와 이미 만들어진 Prompt를 통해서 간단히 챗봇을 구현해봅시다. RetrievalQA.from_chain_type()의 llm 매개변수의 값으로 초기에 임포트한 OpenAI()를 사용할 경우, 기본값으로 ChatGPT API를 사용합니다.**chain_type의 매개변수**의 값으로 "**stuff**"를 사용할 경우, 사용자의 눈에는 보이지 않지만 **자동으로 아래의 프롬프트를 사용**하여 챗봇을 구현합니다.  
```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

- **{텍스트}** 에는 **사용자의 질문으로부터 높은 유사도를 가진 텍스트**가 들어가게 되고,
- **{질문}** 부분은 **사용자의 질문**이 들어가게 됩니다.
- **retriever**는 입력된 사용자의 질문으로부터 유사도를 계산하는 도구를 구현하여 매개변수의 값으로 넘겨주면 됩니다. 앞서 **이미 Chroma를 통해 벡터 도구 객체**로부터 구현한 retreiver를 사용합니다.  

- **return_source_documents**는 **챗봇의 답변에 사용된 텍스트들의 출처를 표시**할 것인지를 의미합니다. return_source_documents의 값을 False로 할 경우, 챗봇의 답변이 어떤 텍스트에 근거하였는지 알 수 없습니다. 여기서는 뒤에서 근거가 되는 텍스트의 출처를 확인할 것이므로 True를 사용합니다.

- **검색 모델** : gpt-3.5-turbo

In [25]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",             # 자동 프롬프트 사용함
    retriever=retriever,            # 앞에서 생성한 벡터DB 검색자
    return_source_documents=True)

## **질의(Query)**

이제 **qa_chain**을 통해 사용자의 입력으로부터 서울 청년 정책과 관련된 챗봇의 답변을 얻을 수 있습니다. 임의의 "대출과 관련된 정책이 궁금합니다"라는 텍스트를 입력하여 qa_chain의 반환 결과를 확인해봅시다.

### **Query → Response**

In [26]:
input_text = "대출과 관련된 정책이 궁금합니다"
chatbot_response = qa_chain(input_text)
print(chatbot_response)
print()
for k, v in chatbot_response.items():
    print(k, ':', v)

/tmp/ipython-input-3407282965.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatbot_response = qa_chain(input_text)


{'query': '대출과 관련된 정책이 궁금합니다', 'result': "서울시에서는 학자금대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 '서울시 학자금대출 신용회복 지원사업'과 청년과 신혼부부의 주거비 부담을 경감하기 위한 '역세권청년주택 주거비지원' 정책을 운영하고 있습니다. 두 정책에 대한 자세한 내용은 각각의 참고사이트를 확인하시면 됩니다.", 'source_documents': [Document(metadata={'source': '35.txt'}, page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n이 정책은 서울시 미래청년기획단이 운영하며, 자세한 내용은 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n신용회복 신청 안내: https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true\n신청사이트: https://youth.seoul.go.kr/site/main/applyLoanCredit/step1'), Document(metadata={'source': '39.txt'}, page_content='정책제목: 역세

qa_chain의 결과를 chatbot_response에 저장하고 출력하면 'query', 'result', source_documents 세 개의 key값을 갖고 있습니다. 이 중 source_documents의 경우 내부적으로 앞서 확인한 형식인 page_content와 meta_data의 값을 가집니다.

'query'의 경우 현재 입력된 사용자의 질문을 의미합니다. 이때 qa_chain은 앞서 구현한 retriever을 통해서  내부적으로 '대출과 관련된 정책이 궁금합니다'와 유사도가 높은 텍스트 2개를 찾아냅니다. metadata 내의 source의 값에 따르면 해당 텍스트는 35.txt 파일과 39.txt 파일의 본문에 해당되며 두  본문(여기서는 page_content의 값)을 통해서 앞서 위에서 확인한 stuff 프롬프트를 통하여 챗봇은 답변을 작성합니다. 해당 챗봇의 **답변**은 결과적으로 '**result**'의 값에 해당됩니다.

현재 chatbot_response의 값은 너무 복잡합니다. 실제 사용자가 사용하는 챗봇을 가정하기 위해 chatbot_response로부터 실제 챗봇 답변에 해당하는 'result' 부분만 꺼내오고, 또한 답변에 사용된 출처를 확인하기 위해 **metadata** 내의 **source**의 값을 추가적으로 출력하는 **get_chatbot_response()** 함수를 구현합니다.

In [27]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])

"신혼 부부의 신혼집 마련을 위한 정책이 있을까?"라는 임의의 입력에 대해서 챗봇의 답변을 확인해보겠습니다.

In [28]:
input_text = "신혼 부부의 신혼집 마련을 위한 정책이 있을까?"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 서울시에서는 '신혼부부 임차보증금 지원'이라는 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련된 것입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서 출처:
23.txt
23.txt


신혼부부 임차 보증금 지원 정책을 소개합니다. 텍스트의 출처가 23.txt가 두 번 출력됩니다. 이는 앞서 text_splitter.split_documents()가 57개의 문서를 64개의 텍스트로 분할하는 가정에서 23.txt가 두 개의 텍스트로 분할되었기 때문입니다. chatbot_response의 값을 출력해보겠습니다.

In [29]:
# print(chatbot_response)
chatbot_response

{'query': '신혼 부부의 신혼집 마련을 위한 정책이 있을까?',
 'result': "네, 서울시에서는 '신혼부부 임차보증금 지원'이라는 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련된 것입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.",
 'source_documents': [Document(metadata={'source': '23.txt'}, page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.'),
  Documen

출처가 23.txt로 동일하지만 두 개의 텍스트는 서로 다릅니다. 원본 파일 23.txt가 두 개의 텍스트로 분할되었기 때문입니다. 또한 두 개의 텍스트에서 **'이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.'** 라는 텍스트가 공통적으로 들어가있음을 확인할 수 있습니다. 이는 RecursiveCharacterTextSplitter() 선언 시에 chunk_overlap을 사용하였기 때문에 텍스트를 분할할 때 겹치지 않도록 분할하는 것이 아니라 일부 겹칠 수 있도록 허용하였기 때문입니다.

### **Query → (Response → Query) → Response**

이번에는 실제 챗봇과의 답변을 가정하고 **사용자의 질문으로부터 챗봇의 답변이 오면 해당 챗봇의 답변으로부터 이어서 사용자가 질문하는 시나리오**를 진행해보겠습니다. "전세 자금이 부족한 사람을 위한 정책 이름이 뭐야?"라는 임의의 질문을 입력합니다.

In [30]:
input_text = "전세자금이 부족한 사람을 위한 정책 이름이 뭐야?"
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

희망두배 청년통장

문서 출처:
40.txt
40.txt


'희망두배 청년통장'이라는 정책이 있음을 알았습니다. 이어서 '희망두배 청년통장'의 구체적인 지원 내용을 문의합니다.

In [31]:
query = "희망두배 청년통장은 어떤걸 지원하니?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장은 교육비, 주거비, 결혼자금, 창업운영자금 등을 위한 저축을 지원하는 정책입니다. 저축 목적에 따라 월 10만원 또는 15만원의 저축액을 선택할 수 있으며, 일정 기간 후에 본인이 적립한 금액의 100%를 지원받을 수 있습니다. 자세한 내용은 서울시청 안심돌봄복지과에서 확인하실 수 있습니다.

문서 출처:
40.txt
36.txt


이번에는 지원 조건을 문의해봅시다.

In [32]:
query = "희망두배 청년통장의 지원 조건은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 지원 조건은 다음과 같습니다:
- 18세에서 34세까지의 청년
- 서울시 거주자
- 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우

문서 출처:
40.txt
36.txt


희망두배 청년통장의 마감 기한을 문의해봅시다.

In [33]:
query = "희망두배 청년통장의 마감 기한은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 마감 기한은 서울시복지재단, 서울시 및 25개 자치구 홈페이지에서 확인하실 수 있습니다. 해당 사이트에서 신청 기한을 확인하시는 것이 좋습니다.

문서 출처:
40.txt
37.txt


원본 문서에서 마감 기한이 구체적으로 나와있지 않아 마감 기한을 알 수 없다고 답변합니다. 해당 챗봇은 주어진 57개의 문서를 기반으로 답변하기 때문입니다. 이렇게 다수의 문서로부터 질의 응답을 할 수 있는 챗봇을 구현해보았습니다. 이렇게 구현한 챗봇을 앞서 실습과 같이 streamlit으로 UI를 구현하거나, 카카오톡이나 텔레그램 등을 연동하여 나만의 커스텀 챗봇을 구현할 수 있습니다.

## **gradio UI Interface**

- ML 데모를 만드는 가장 쉬운 방법
- AI 연구자들은 모델을 만들고 논문을 쓰는 데는 익숙했지만, 이 모델이 실제로 어떻게 작동하는지 보여주는 '데모'를 만드는 데는 어려움을 겪었음
- 2020년, Abubakar Abid를 비롯한 연구팀이 이러한 문제를 해결하기 위해 Gradio를 개발
- 2021년, Hugging Face는 Gradio를 인수
- HF Spaces(모델 데모를 호스팅하는 플랫폼)의 표준 UI 빌더
- **주요 기능**
    - **(극도의 단순함)** : **gr.Interface(...)** 몇 줄이면 UI가 완성
    - **(다양한 컴포넌트)** : 텍스트 박스, 이미지 업로드, 슬라이더, 드롭다운, 오디오 녹음, 비디오 등 AI 모델에 필요한 거의 모든 입출력 컴포넌트를 미리 제공
    - **(자동 공유 링크)** :  **.launch(share=True) 옵션** 하나만 추가하면, 방화벽이나 복잡한 설정 없이도 72시간 동안 유효한 공개 URL을 생성
    - **(유연성 및 확장성)** : **gr.Blocks**라는 기능을 통해 챗봇 UI, 탭(Tab) 구분, 복잡한 레이아웃 등 고도로 커스터마이징된 웹 앱도 구축할 수 있음
    - **(임베딩)** : Jupyter Notebook, Google Colab, 웹사이트 등 어디에나 쉽게 데모 창을 삽입할 수 있음


In [ ]:
!pip install gradio

To create a public link, set `share=True` in `launch()`.  
Running on https://localhost:7860/  

라는 식의 문구가 나오면 위의 localhost로 시작하는 주소를 클릭하세요.

In [ ]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += ' # sources :'

      # 답변의 출처를 표기
      for i, doc in enumerate(result['source_documents']):
          bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

/tmp/ipython-input-1083721868.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2666a9609dc55603d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2666a9609dc55603d9.gradio.live
